# Analysis of Cell Phone Reviews

***************************************************

## Importing required libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Importing and cleaning the dataset

In [2]:
df = pd.read_csv('data/20191226-reviews.csv')

In [3]:
df.head()

,asin,name,rating,date,verified,title,body,helpfulVotes
0,B0000SX2UC,Janet,3,"October 11, 2005",False,"Def not best, but not worst",I had the Samsung A600 for awhile which is abs...,1.0
1,B0000SX2UC,Luke Wyatt,1,"January 7, 2004",False,Text Messaging Doesn't Work,Due to a software issue between Nokia and Spri...,17.0
2,B0000SX2UC,Brooke,5,"December 30, 2003",False,Love This Phone,"This is a great, reliable phone. I also purcha...",5.0
3,B0000SX2UC,amy m. teague,3,"March 18, 2004",False,"Love the Phone, BUT...!","I love the phone and all, because I really did...",1.0
4,B0000SX2UC,tristazbimmer,4,"August 28, 2005",False,"Great phone service and options, lousy case!",The phone has been great for every purpose it ...,1.0


In [4]:
df.shape

(67986, 8)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67986 entries, 0 to 67985
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   asin          67986 non-null  object 
 1   name          67984 non-null  object 
 2   rating        67986 non-null  int64  
 3   date          67986 non-null  object 
 4   verified      67986 non-null  bool   
 5   title         67972 non-null  object 
 6   body          67965 non-null  object 
 7   helpfulVotes  27215 non-null  float64
dtypes: bool(1), float64(1), int64(1), object(5)
memory usage: 3.7+ MB


In [6]:
df.isnull().sum()

asin                0
name                2
rating              0
date                0
verified            0
title              14
body               21
helpfulVotes    40771
dtype: int64

- We have $14$ null values in the rating column of our dataset.
- We have to drop the null values.

In [7]:
title_miss = df[df.title.isnull()]

In [8]:
title_miss

,asin,name,rating,date,verified,title,body,helpfulVotes
30010,B01NB1KG8U,Sylvester Ofosuhene,5,"December 24, 2019",True,NaN,NaN,NaN
30949,B06XR1K6HR,MOHAMED ALI,5,"January 17, 2019",True,NaN,Almost like pretty new,NaN
32883,B06XSF5C42,Candice,5,"June 13, 2019",True,NaN,Love this phone. Everything's worked great. So...,NaN
35016,B071H9KKKF,Wauany,5,"June 17, 2018",True,NaN,Like the phone so far!!! Never had an expensiv...,NaN
42935,B077T4MVZ6,Evaldina,4,"November 14, 2018",True,NaN,Love it,NaN
45899,B079X7DQ4Q,Roberto,5,"November 25, 2019",True,NaN,NaN,NaN
45905,B079X7DQ4Q,Mahmood al rahawi,5,"December 7, 2018",True,NaN,"I get that phone I needed ,, thanks .",NaN
46470,B07BHT4KGM,Roberto,5,"November 25, 2019",True,NaN,NaN,NaN
46476,B07BHT4KGM,Mahmood al rahawi,5,"December 7, 2018",True,NaN,"I get that phone I needed ,, thanks .",NaN
50404,B07FZH9BGV,Henry,5,"November 1, 2018",True,NaN,Great phone...A++,1.0


In [9]:
title_miss[['rating', 'title', 'body']]

,rating,title,body
30010,5,NaN,NaN
30949,5,NaN,Almost like pretty new
32883,5,NaN,Love this phone. Everything's worked great. So...
35016,5,NaN,Like the phone so far!!! Never had an expensiv...
42935,4,NaN,Love it
45899,5,NaN,NaN
45905,5,NaN,"I get that phone I needed ,, thanks ."
46470,5,NaN,NaN
46476,5,NaN,"I get that phone I needed ,, thanks ."
50404,5,NaN,Great phone...A++


- If we closely look, we can find that all the **missing titles** are associated with very high rating in the *rating* column.
- The *body* column for each of the **missing titles** also suggest that the customers are satisfied with the purchase.

> So it is safe to say that missing titles can be associated with high rating.<br>
> However, we cannot use missing titles to train our model so we have to drop the missing titles.

In [10]:
dataset = df[~df.title.isnull()]

In [11]:
dataset.reset_index(inplace = True)

In [12]:
dataset.isnull().sum()

index               0
asin                0
name                2
rating              0
date                0
verified            0
title               0
body               16
helpfulVotes    40758
dtype: int64

Now we can see that the dataset does not contain any missing **title** values.

- Now we can choose the *Independent* and *Dependent* variables.
- We can use the *title* column as our **Reviews**(*independent variable*) and <br> *rating* columns as our **Rating**(*dependent variables*)

In [13]:
reviews = dataset.title
ratings = dataset.rating

- If the rating is $\gt=3$ then it is a positive rating, else it is a negative rating.

In [14]:
ratings = np.where(ratings >= 3, 1, 0)

In [15]:
dataset.shape

(67972, 9)

In [16]:
reviews.head()

0                     Def not best, but not worst
1                     Text Messaging Doesn't Work
2                                 Love This Phone
3                         Love the Phone, BUT...!
4    Great phone service and options, lousy case!
Name: title, dtype: object

 ## Text Preprocessing

### Import required libraries

In [17]:
import nltk #Natural Language Toolkit library
from nltk.corpus import stopwords #Library to remove stopwords
from nltk.stem.porter import PorterStemmer #Library to stem words
import re 

In [18]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\Aditya
[nltk_data]     Dash\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [19]:
ps = PorterStemmer()

In [20]:
m = len(reviews)

In [21]:
data = []
for i in range(m):
    review = reviews[i]
    review = re.sub('[^a-zA-Z]', ' ', review)
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if word not in set(stopwords.words('english'))]
    review = ' '.join(review)
    data.append(review)

### Creating bag of words model

In [22]:
from sklearn.feature_extraction.text import CountVectorizer

In [23]:
cv = CountVectorizer()

In [24]:
X = cv.fit_transform(data).toarray()

## Creating train and test set

In [25]:
from sklearn.model_selection import train_test_split

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, ratings.ravel(), test_size = 0.25, random_state = 42)

In [27]:
max_features = X.shape[-1]

In [28]:
max_features

6984

# Model Building

## Libraries to build model

In [29]:
import tensorflow.keras as keras

In [51]:
model = keras.Sequential() #Model Instantiation

In [52]:
model.add(keras.layers.Input(shape = (max_features,))) #Input Layer

In [53]:
model.add(keras.layers.Dense(units = 200, activation = 'relu')) #Hidden Layer
model.add(keras.layers.Dropout(rate = 0.8))

In [54]:
model.add(keras.layers.Dense(units = 1, activation = 'sigmoid')) #Output Layer

In [55]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [56]:
model_checkpoint = keras.callbacks.ModelCheckpoint('sentiment.h5', save_best_only = True)

In [57]:
history = model.fit(X_train, y_train, batch_size = 32, epochs = 100, validation_split = 0.2, callbacks = [model_checkpoint])

Train on 40783 samples, validate on 10196 samples
Epoch 1/100
40783/40783 [==============================] - 6s 149us/sample - loss: 0.3654 - accuracy: 0.8378 - val_loss: 0.3130 - val_accuracy: 0.8668
Epoch 2/100
40783/40783 [==============================] - 6s 136us/sample - loss: 0.2983 - accuracy: 0.8749 - val_loss: 0.3078 - val_accuracy: 0.8699
Epoch 3/100
40783/40783 [==============================] - 5s 134us/sample - loss: 0.2810 - accuracy: 0.8840 - val_loss: 0.3078 - val_accuracy: 0.8680
Epoch 4/100
40783/40783 [==============================] - 6s 136us/sample - loss: 0.2706 - accuracy: 0.8901 - val_loss: 0.3080 - val_accuracy: 0.8700
Epoch 5/100
40783/40783 [==============================] - 6s 143us/sample - loss: 0.2585 - accuracy: 0.8967 - val_loss: 0.3089 - val_accuracy: 0.8705
Epoch 6/100
40783/40783 [==============================] - 6s 143us/sample - loss: 0.2504 - accuracy: 0.8990 - val_loss: 0.3117 - val_accuracy: 0.8707
Epoch 7/100
40783/40783 [===================

40783/40783 [==============================] - 6s 138us/sample - loss: 0.1583 - accuracy: 0.9396 - val_loss: 0.4659 - val_accuracy: 0.8751
Epoch 54/100
40783/40783 [==============================] - 5s 135us/sample - loss: 0.1568 - accuracy: 0.9398 - val_loss: 0.4653 - val_accuracy: 0.8753
Epoch 55/100
40783/40783 [==============================] - 6s 135us/sample - loss: 0.1585 - accuracy: 0.9396 - val_loss: 0.4646 - val_accuracy: 0.8747
Epoch 56/100
40783/40783 [==============================] - 6s 138us/sample - loss: 0.1586 - accuracy: 0.9396 - val_loss: 0.4722 - val_accuracy: 0.8747
Epoch 57/100
40783/40783 [==============================] - 6s 141us/sample - loss: 0.1559 - accuracy: 0.9417 - val_loss: 0.4764 - val_accuracy: 0.8743
Epoch 58/100
40783/40783 [==============================] - 6s 140us/sample - loss: 0.1540 - accuracy: 0.9415 - val_loss: 0.4812 - val_accuracy: 0.8746
Epoch 59/100
40783/40783 [==============================] - 8s 184us/sample - loss: 0.1557 - accuracy

In [58]:
model.evaluate(X_test, y_test)

16993/16993 [==============================] - 2s 102us/sample - loss: 0.6160 - accuracy: 0.8772


[0.6160465055736323, 0.87718475]

In [59]:
best = keras.models.load_model('sentiment.h5')

In [60]:
best.evaluate(X_test, y_test)

16993/16993 [==============================] - 2s 89us/sample - loss: 0.2992 - accuracy: 0.8741


[0.29922819589308264, 0.87412465]

## Save CountVectorizer object

In [61]:
import pickle

In [62]:
with open('countvectorizer', 'wb') as fout:
    pickle.dump(cv, fout)